## Calculate irradiance

In [32]:
irrad_expr = mms.sphere_integral(
    sol_expr,
    angle=angle
)

irrad_sym = mms.symify(irrad_expr, *space, **param_vals)
irrad_func_N = kelp_compute.sym_to_num(irrad_sym, *space)

## Numerical Solution

In [ ]:
# Grid options
ns = 16
nz = 16
na = 10
nomega = na*(na-2) + 2

# Domain size
rope_spacing = 1
zmax = 1

# Solver options
lis_opts = "-i gmres -restart 100 -tol 1e-3"
num_scatters = 1

# Numerical function for source expansion
source_expansion_N = gen_series_N(source_expr, num_scatters, **param_vals)

# Calculate leading order solution
_, noscat_results = kelp_compute.solve_rte_with_callbacks_full(
    ns, nz, na,
    rope_spacing, zmax,
    b, abs_sym, source_sym, source_expansion_N, bc_sym, vsf_sym,
    num_scatters=0, fd_flag=False, lis_opts=lis_opts
)

# Calculate asymptotic solution
_, asymptotic_results = kelp_compute.solve_rte_with_callbacks_full(
    ns, nz, na,
    rope_spacing, zmax,
    b, abs_sym, source_sym, source_expansion_N, bc_sym, vsf_sym,
    num_scatters=num_scatters, fd_flag=False, lis_opts=lis_opts
)

# Calculate finite difference solution
_, fd_results = kelp_compute.solve_rte_with_callbacks_full(
    ns, nz, na,
    rope_spacing, zmax,
    b, abs_sym, source_sym, source_expansion_N, bc_sym, vsf_sym,
    num_scatters=0, fd_flag=True, lis_opts=lis_opts
)

# Extract numerical solutions
noscat_sol = noscat_results['rad']
asymptotic_sol = asymptotic_results['rad']
fd_sol = fd_results['rad']

# Evaluate true solution on same grid
x, y, z, theta, phi = gen_grid(ns, nz, na, rope_spacing, zmax)
true_sol = sol_func_N(x, y, z, theta, phi)
true_irrad = irrad_func_N(x, y, z)[:,:,:,0]

## Symbolic Solution (no scattering)

In [ ]:
one_angle_dict = {
    'theta': th,
    'phi': ph
}
one_angle_tuple = (th, ph)

In [ ]:
# Declare variables
s, s_p, s_pp = sp.var('s, s_p, s_{pp}')
x0, y0 = sp.var('x_0, y_0')
vec_x0 = sp.Matrix([x0, y0, 0])

In [ ]:
do_symbolic = True
num_sym_scat = 1
if do_symbolic:
    # Calculate absorption and source
    # along ray path
    a_tilde = abs_sym(
        *sp.simplify(
            subs_dict(
                vec_l0(vec_x0, vec_om, s_pp, zmax),
                one_angle_dict
            )
        )
    )
    sigma_tilde = source_sym(
        *sp.simplify(
            subs_dict(
                vec_l0(vec_x0, vec_om, s_p, zmax),
                one_angle_dict
            )
        ), 
        *one_angle_tuple
    )

    # Integrate light from distributed source
    u0_source_expr = sp.integrate(
        sigma_tilde * sp.exp(
            -sp.integrate(
                a_tilde,
                (s_pp, s_p, s)
            )
        ),
        (s_p, 0, s)
    )

    # Integrate light from boundary condition
    u0_bc_expr = (
        bc_sym(*angle) * sp.exp(
            -sp.integrate(
                a_tilde,
                (s_pp, 0, s)
            )
        )
    )

    # Superpose source and bc solutions
    u0_s_expr = u0_source_expr + u0_bc_expr

    # Convert to funcion of x, y, z
    u0_func = sp.lambdify(
        ('s', 'x_0', 'y_0'),
        u0_s_expr,
        modules=("sympy",)
    )
    L_symbolic_expr = u0_func(
        sp.Symbol('z')*sp.sec(ph),
        sp.Symbol('x') - sp.Symbol('z')*sp.tan(ph)*sp.cos(th),
        sp.Symbol('y') - sp.Symbol('z')*sp.tan(ph)*sp.sin(th),
    )

    # Convert to numerical function
    L_symbolic_N = sp.lambdify(
        space,
        L_symbolic_expr, 
    )

    # Evaluate on discrete grid
    symbolic_sol = L_symbolic_N(x, y, z)

## Plot

## 1D Slices

In [ ]:
plot_inds = np.zeros_like(true_sol, dtype=bool)
plot_inds[0,0,:,p] = True

# Source & abs
plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
plt.plot(z[plot_inds], abs_func_N(x,y,z)[plot_inds], 'C0o-', label='abs')
plt.plot(z[plot_inds], source_func_N(x,y,z,theta,phi)[plot_inds], 'C1o-', label='source')
plt.xlabel('z')
plt.legend()

# Solution
plt.subplot(2,2,2)
plt.plot(z[plot_inds], true_sol[plot_inds], 'C2o-', label='true')
plt.plot(z[plot_inds], noscat_sol[plot_inds], 'C3o-', label='noscat')
plt.plot(z[plot_inds], asymptotic_sol[plot_inds], 'C4o-', label='asym')
plt.plot(z[plot_inds], fd_sol[plot_inds], 'C5o-', label='fd')
plt.plot(z[plot_inds], symbolic_sol[plot_inds], 'C6o-', label='symbolic')
plt.xlabel('z')
plt.ylabel('sol')
plt.legend()

# Error
plt.subplot(2,2,3)
plt.plot(z[plot_inds], -true_sol[plot_inds]+true_sol[plot_inds], 'C2o-', label='true')
plt.plot(z[plot_inds], -true_sol[plot_inds]+noscat_sol[plot_inds], 'C3o-', label='noscat')
plt.plot(z[plot_inds], -true_sol[plot_inds]+asymptotic_sol[plot_inds], 'C4o-', label='asym')
plt.plot(z[plot_inds], -true_sol[plot_inds]+fd_sol[plot_inds], 'C5o-', label='fd')
plt.plot(z[plot_inds], -true_sol[plot_inds]+symbolic_sol[plot_inds], 'C6o-', label='symbolic')
plt.xlabel('z')
plt.ylabel('sol - true')
plt.legend()

# Log Error
plt.subplot(2,2,4)
plt.plot(z[plot_inds], np.log(np.abs(-true_sol[plot_inds]+noscat_sol[plot_inds])), 'C3o-', label='noscat')
plt.plot(z[plot_inds], np.log(np.abs(-true_sol[plot_inds]+asymptotic_sol[plot_inds])), 'C4o-', label='asym')
plt.plot(z[plot_inds], np.log(np.abs(-true_sol[plot_inds]+fd_sol[plot_inds])), 'C5o-', label='fd')
plt.plot(z[plot_inds], np.log(np.abs(-true_sol[plot_inds]+symbolic_sol[plot_inds])), 'C6o-', label='symbolic')
plt.xlabel('z')
plt.ylabel('sol - true')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
np.abs(true_sol - symbolic_sol).max()

## 2D Slices

In [ ]:
plt.figure()
plt.contourf(x[:,0,:,0], z[:,0,:,0], true_sol[:,0,:,0])
plt.colorbar()
plt.title('true')
plt.xlabel('x')
plt.ylabel('z')

plt.figure()
plt.contourf(x[:,0,:,0], z[:,0,:,0], fd_sol[:,0,:,0])
plt.colorbar()
plt.title('fd')
plt.xlabel('x')
plt.ylabel('z')

plt.figure()
plt.contourf(x[:,0,:,0], z[:,0,:,0], noscat_sol[:,0,:,0])
plt.colorbar()
plt.title('noscat')
plt.xlabel('x')
plt.ylabel('z')

plt.figure()
plt.contourf(x[:,0,:,0], z[:,0,:,0], asymptotic_sol[:,0,:,0])
plt.colorbar()
plt.title('asymptotic')
plt.xlabel('x')
plt.ylabel('z')
plt.show()

## Volume Plots

### True

In [ ]:
discrete_plot.volshow_zoom_correct_scale(x[:,0,0,0], y[0,:,0,0], z[0,0,:,0], true_irrad)

### FD

In [ ]:
discrete_plot.volshow_zoom_correct_scale(x[:,0,0,0], y[0,:,0,0], z[0,0,:,0], fd_results['irrad'])

### Noscat

In [ ]:
discrete_plot.volshow_zoom_correct_scale(x[:,0,0,0], y[0,:,0,0], z[0,0,:,0], noscat_results['irrad'])

### Asymptotics

In [ ]:
discrete_plot.volshow_zoom_correct_scale(x[:,0,0,0], y[0,:,0,0], z[0,0,:,0], asymptotic_results['irrad'])